In [7]:
import sys 

sys.path.append('../lib/nameh-khan')

## Import 

In [9]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

from nameh_khan.page_reader.utils.segmentation import SimpleSegmentation

## Data Path

In [13]:
DATASET_DIR = 'raw-dataset/nazanin/00001'

RAW_IMAGE_PATH = '{}/page.png'.format(DATASET_DIR)
RAW_LABEL_PATH = '{}/label.csv'.format(DATASET_DIR)

## Data Preparation 

In [22]:
image = cv2.imread(RAW_IMAGE_PATH, flags=cv2.IMREAD_GRAYSCALE)
segmentation = SimpleSegmentation()
box_locations = segmentation.transform(image)
label_pdf = (
    pd.DataFrame(box_locations)
    .sort_values(by=['y1'], ascending=[True])
    .assign(lineChange=lambda x: (x.y1 > x.y2.shift(1)).astype(int))
    .assign(lineNumber=lambda x: x.lineChange.cumsum())
    .assign(word = '')
    .sort_values(by=['lineNumber', 'x2'], ascending=[True, False])
    .reset_index(drop=True)
    .loc[:, ['word', 'x1', 'y1', 'x2', 'y2']]
)

## IO

In [24]:
label_pdf.to_csv(RAW_LABEL_PATH, index=False)